# Image Classification using `sklearn.svm`

## Libraries

In [10]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm, metrics, datasets
from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split

from skimage.io import imread
from skimage.transform import resize

import time
import datetime

# My addition to the original code
import os
from skimage.feature import hog
from random import randint

import os
from skimage.util import img_as_ubyte
from skimage import io
from skimage.filters import threshold_otsu


In [11]:
# Start point
start_time = time.time()
print("Start time: ", datetime.datetime.now())

Start time:  2019-01-09 11:23:46.845579


## Preprocessing

In [12]:
def HOG(image):
    img = hog(image,
              orientations=8,
              pixels_per_cell=(4, 4),
              cells_per_block=(2, 2),
              block_norm='L2-Hys',
              multichannel = False,
              feature_vector = True)
    return img

In [13]:
# binarization using Otsu's method
def binarize(inp_image):
    thresh = threshold_otsu(inp_image)
    binary_thresh_img = inp_image > thresh

    img_binary = img_as_ubyte(binary_thresh_img)

    return img_binary

In [14]:
# defining things for the morphological filtering

from skimage.morphology import erosion, dilation, opening, closing, white_tophat
from skimage.morphology import black_tophat, skeletonize, convex_hull_image
from skimage.morphology import disk, square, diamond

selem = disk(1)

## Load images in structured directory

In [23]:
# Function that builds pair of signatures on which the algorithm is going to work.

def match_2(flat_data, target, n_pairs):
    """
    Parameters

    - flat_data: a list of ndarray containing the flattern information coming from
      the hog on the signature image.
    - target: a list of strings. Each component can be 'Genuine' of 'Forgery'.

    Returns

    A dataset divided in:
    - data: a list of ndarray, each one originating from the concatenation of 2 array of the input flat_data.
    - target: a list of strings, each one deriving from the concatenation of 2 strings of input target list.

    -------------------------------------------------------

    """

    
    m_flat_data=[]
    m_target=[]
    g_indexes=[ind for ind, t in enumerate(target) if t=='Genuine']
    f_indexes=[ind for ind, t in enumerate(target) if t=='Forgery']
    matches=[]
    for i in range(0,n_pairs): # creates n pairs: Genuine - Genuine
        n=randint(0,len(g_indexes)-1)
        m=randint(0,len(g_indexes)-1)
        while n == m or (g_indexes[n],g_indexes[m]) in matches:
            n=randint(0,len(g_indexes)-1)
            m=randint(0,len(g_indexes)-1)
        m_flat_data.append(np.concatenate((flat_data[g_indexes[n]],flat_data[g_indexes[m]])))
        m_target.append(target[g_indexes[n]]+' '+target[g_indexes[m]])
        matches.append((g_indexes[n],g_indexes[m]))


    matches=[]
    for i in range(0,n_pairs): # Creates n pairs matchings: Genuine - Forgery
        n=randint(0,len(g_indexes)-1)
        m=randint(0,len(f_indexes)-1)
        while (g_indexes[n], f_indexes[m]) in matches:
            n=randint(0,len(g_indexes)-1)
            m=randint(0,len(f_indexes)-1)
        m_flat_data.append(np.concatenate((flat_data[g_indexes[n]],flat_data[f_indexes[m]])))
        m_target.append(target[g_indexes[n]]+' '+target[f_indexes[m]])
        matches.append((g_indexes[n], f_indexes[m]))
   


    return Bunch(data=m_flat_data,
                 target=m_target
                )


In [17]:
def load_image_files(container_path, dimension=(100, 144)): # height x lenght
    """
    Load image files with categories as subfolder names
    which performs like scikit-learn sample dataset

    Parameters
    ----------
    container_path : string or unicode
        Path to the main folder holding one subfolder per category
    dimension : tuple
        size to which image are adjusted to

    Returns
    -------
    Bunch
    """
    image_dir = Path(container_path)
    subj = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = ['Genuine', 'Forgery']

    descr = "An image classification dataset"
    #images = []
    flat_data = []
    target = []

    for n, s in enumerate(subj):
        folders = [sub_directory for sub_directory in s.iterdir() if sub_directory.is_dir()]
        temp_flat_data=[] # Temporary list of flat_data used to store the information of the current subject
                          # and reinitialized as empty list for each subject
        temp_images=[] # Temporary list of images
        temp_target=[] # Temporary list of target
        for sub_dir in folders:
            if sub_dir.name in categories: # Skip the Disguised folder
                for file in sub_dir.iterdir():
                    img_gray = img_as_ubyte(io.imread(file, as_gray=True)) # load the images in grayscale
                    img_bin = binarize(img_gray)
                    img_resized = resize(img_bin, dimension, anti_aliasing=True, mode='reflect')


                    temp_flat_data.append(HOG(img_resized))

                    # NOTE: resize() produces a dtype('float64') while imread() and binarize() return dtype('uint8')
                    # the opening filter is bad

                    temp_target.append(sub_dir.name)

        #subj_dataset = match_2(temp_flat_data, temp_target, 5) #now the number of couples can be setted

        subj_dataset = match_2(temp_flat_data, temp_target, 5) 
        flat_data += subj_dataset.data
        target += subj_dataset.target

    flat_data = np.array(flat_data)
    target = np.array(target)

    return Bunch(data=flat_data,
                 target=target,
                 target_names=categories,
                 DESCR=descr)

In [18]:
# To let the code run it is necessary to put this script in the same folder which contains the database folder

db_folder='Database' # Modify with the name of the folder containing the database.
image_dataset = load_image_files(os.path.join(os.getcwd(),db_folder))

print("----Dataset Loaded----")

----Dataset Loaded----


### Split data in Training Set and Testing Set

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    image_dataset.data, image_dataset.target, test_size=0.3,random_state=109)

print("----Dataset Splitted----")

----Dataset Splitted----


In [20]:
print("Training Set shape: {}".format(X_train.shape))
print("Test Set shape: {}".format(X_test.shape))

print("Training Set shape: {}".format(y_train.shape))
print("Test Set shape: {}".format(y_test.shape))

Training Set shape: (21014, 53760)
Test Set shape: (9006, 53760)
Training Set shape: (21014,)
Test Set shape: (9006,)


### Train data with parameter optimization

In [21]:
print("----Training----")

param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
svc = svm.SVC()
clf = GridSearchCV(svc, param_grid, cv=3)
clf.fit(X_train, y_train)

print("----Training Ended----")

----Training----


MemoryError: 

### Predict

In [ ]:
y_pred = clf.predict(X_test)

### Report

In [ ]:
print("Classification report for - \n{}:\n{}\n".format(
    clf, metrics.classification_report(y_test, y_pred)))

In [ ]:
# End point
end_time = time.time()

uptime = end_time - start_time

human_uptime = datetime.timedelta(seconds=uptime)

print("End time: ", datetime.datetime.now())
print("Uptime :" ,human_uptime)